In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원 축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# 학습 모델 저장을 위한 라이브러리
import pickle

In [2]:
# 데이터를 읽어온다.
df1 = pd.read_parquet(r'data/1.회원정보_train_filtered_data.parquet')
df2 = pd.read_parquet(r'data/신용정보_VIF용5.parquet')
df3 = pd.read_parquet(r'data/승인매출정보_변수추출_1차.parquet')
df4 = pd.read_parquet(r'data/4.청구입금정보(train).parquet')
df5 = pd.read_parquet(r'data/5.잔액정보(train).parquet')
df6 = pd.read_parquet(r'data/6.채널정보_VIF용2.parquet')

In [3]:
# 1. 먼저 기준이 되는 df1부터 시작
merged_df = df1.copy()

# 2. ID & 기준년월 기준으로 순차 병합
merged_df = merged_df.merge(df2, on=['ID', '기준년월'], how='inner')
merged_df = merged_df.merge(df3, on=['ID', '기준년월'], how='inner')
merged_df = merged_df.merge(df4, on=['ID', '기준년월'], how='inner')
merged_df = merged_df.merge(df5, on=['ID', '기준년월'], how='inner')
merged_df = merged_df.merge(df6, on=['ID', '기준년월'], how='inner')


In [4]:
merged_df

,기준년월,ID,이용가능여부_해외겸용_본인,보유여부_해외겸용_본인,수신거부여부_TM,수신거부여부_메일,수신거부여부_DM,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,...,청구금액_R3M,청구금액_B0,할인건수_R3M,월중평잔_일시불_B0M,월중평잔_일시불,평잔_일시불_3M,잔액_일시불_B0M,잔액_일시불_B1M,잔액_일시불_B2M,인입횟수_ARS_R6M
0,201807,TRAIN_000000,0,0,0,0,0,196,196,3681,...,46588,12226,1회 이상,1084,1503,1791,998,736,1083,10회 이상
1,201807,TRAIN_000001,0,0,0,0,0,13475,13475,13323,...,10530,5834,1회 이상,4090,4447,3761,2565,6795,1424,1회 이상
2,201807,TRAIN_000002,1,1,0,0,0,23988,23988,24493,...,85931,21866,1회 이상,5006,5540,6796,5312,5150,6143,1회 이상
3,201807,TRAIN_000003,1,1,0,0,0,3904,3904,5933,...,61518,16356,1회 이상,487,606,772,730,462,708,10회 이상
4,201807,TRAIN_000004,1,1,0,0,0,1190,0,0,...,0,0,1회 이상,0,0,0,0,0,0,1회 이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,1,1,0,0,0,10755,7267,5640,...,0,0,1회 이상,0,0,0,0,0,0,1회 이상
2399996,201812,TRAIN_399996,1,1,0,0,0,27636,27636,26357,...,37515,14402,1회 이상,4412,5515,9424,3351,4142,4362,1회 이상
2399997,201812,TRAIN_399997,1,1,1,1,1,23187,23187,17171,...,22274,5731,1회 이상,2694,3046,2998,2524,1523,2898,1회 이상
2399998,201812,TRAIN_399998,0,0,1,1,1,0,0,0,...,0,0,1회 이상,0,0,0,0,0,0,1회 이상


In [5]:
train_df2=merged_df.copy()

train_df2['이용금액대'] = train_df2['이용금액대'].factorize()[0]
train_df2['이용가능여부_해외겸용_본인'] = train_df2['이용가능여부_해외겸용_본인'].factorize()[0]
train_df2['보유여부_해외겸용_본인'] = train_df2['보유여부_해외겸용_본인'].factorize()[0]
train_df2['상향가능한도금액'] = train_df2['상향가능한도금액'].factorize()[0]
train_df2['상향가능CA한도금액'] = train_df2['상향가능CA한도금액'].factorize()[0]
train_df2['할인건수_R3M'] = train_df2['할인건수_R3M'].factorize()[0]
train_df2['인입횟수_ARS_R6M'] = train_df2['인입횟수_ARS_R6M'].factorize()[0]
train_df2['수신거부여부_TM'] = train_df2['수신거부여부_TM'].factorize()[0]
train_df2['수신거부여부_메일'] = train_df2['수신거부여부_메일'].factorize()[0]
train_df2['수신거부여부_DM'] = train_df2['수신거부여부_DM'].factorize()[0]
train_df2

,기준년월,ID,이용가능여부_해외겸용_본인,보유여부_해외겸용_본인,수신거부여부_TM,수신거부여부_메일,수신거부여부_DM,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,...,청구금액_R3M,청구금액_B0,할인건수_R3M,월중평잔_일시불_B0M,월중평잔_일시불,평잔_일시불_3M,잔액_일시불_B0M,잔액_일시불_B1M,잔액_일시불_B2M,인입횟수_ARS_R6M
0,201807,TRAIN_000000,0,0,0,0,0,196,196,3681,...,46588,12226,0,1084,1503,1791,998,736,1083,0
1,201807,TRAIN_000001,0,0,0,0,0,13475,13475,13323,...,10530,5834,0,4090,4447,3761,2565,6795,1424,1
2,201807,TRAIN_000002,1,1,0,0,0,23988,23988,24493,...,85931,21866,0,5006,5540,6796,5312,5150,6143,1
3,201807,TRAIN_000003,1,1,0,0,0,3904,3904,5933,...,61518,16356,0,487,606,772,730,462,708,0
4,201807,TRAIN_000004,1,1,0,0,0,1190,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,1,1,0,0,0,10755,7267,5640,...,0,0,0,0,0,0,0,0,0,1
2399996,201812,TRAIN_399996,1,1,0,0,0,27636,27636,26357,...,37515,14402,0,4412,5515,9424,3351,4142,4362,1
2399997,201812,TRAIN_399997,1,1,1,1,1,23187,23187,17171,...,22274,5731,0,2694,3046,2998,2524,1523,2898,1
2399998,201812,TRAIN_399998,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 8월

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

train_df4 = train_df2[train_df2['기준년월'] == 201808]
train_df4

,기준년월,ID,이용가능여부_해외겸용_본인,보유여부_해외겸용_본인,수신거부여부_TM,수신거부여부_메일,수신거부여부_DM,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,이용카드수_신용체크,_2순위카드이용금액,_1순위카드이용건수,_2순위카드이용건수,이용카드수_신용,상향가능한도금액,상향가능CA한도금액,정상청구원금_B5M,정상청구원금_B0M,정상청구원금_B2M,이용금액_일시불_R12M,이용금액_일시불_B0M,이용금액_오프라인_B0M,이용금액_일시불_R6M,이용금액_일시불_R3M,정상입금원금_B5M,정상입금원금_B0M,이용금액_오프라인_R3M,이용금액_오프라인_R6M,정상입금원금_B2M,_3순위업종_이용금액,_2순위업종_이용금액,이용건수_신용_R12M,_2순위쇼핑업종_이용금액,최대이용금액_일시불_R12M,이용건수_신판_R12M,이용건수_일시불_R12M,_1순위업종_이용금액,_3순위쇼핑업종_이용금액,이용가맹점수,이용건수_오프라인_B0M,이용건수_오프라인_R6M,이용건수_오프라인_R3M,쇼핑_도소매_이용금액,이용건수_신용_R6M,이용건수_신용_B0M,이용건수_신용_R3M,이용건수_신판_R6M,이용건수_신판_B0M,이용건수_신판_R3M,이용건수_일시불_R6M,이용건수_일시불_B0M,이용건수_일시불_R3M,_1순위교통업종_이용금액,연체입금원금_B0M,쇼핑_마트_이용금액,쇼핑_슈퍼마켓_이용금액,교통_주유이용금액,이용금액_온라인_B0M,연체입금원금_B5M,연체입금원금_B2M,이용금액_페이_온라인_B0M,_1순위쇼핑업종_이용금액,연속유실적개월수_기본_24M_카드,이용금액대,청구금액_R6M,청구금액_R3M,청구금액_B0,할인건수_R3M,월중평잔_일시불_B0M,월중평잔_일시불,평잔_일시불_3M,잔액_일시불_B0M,잔액_일시불_B1M,잔액_일시불_B2M,인입횟수_ARS_R6M
400000,201808,TRAIN_000000,0,0,0,0,0,-1190,-1190,2529,1,0,23,0,1,0,0,15593,15008,15712,22754,905,3883,7041,1250,11161,7685,4852,13568,15592,734,1523,159,0,5350,140,137,2000,0,5,5,52,17,0,74,7,23,64,5,18,62,5,17,204,7322,0,0,0,0,4431,120,0,0,13,0,119419,53336,15756,0,1092,1388,1736,780,997,736,0
400001,201808,TRAIN_000001,0,0,0,0,0,13130,13130,13300,1,0,48,0,1,2,1,3462,3117,2620,52593,3047,4119,20557,14062,2799,3117,15654,16305,2620,1798,2206,193,455,12485,191,191,2504,313,33,14,91,53,662,73,14,41,73,14,41,73,14,41,1281,0,0,455,1281,0,662,0,0,662,12,1,14626,9194,3209,0,2850,3096,4643,2898,2571,6795,1
400002,201808,TRAIN_000002,1,1,0,0,0,24421,24421,25118,1,0,51,0,1,0,0,23327,22547,24649,54220,6662,9989,38662,18936,14538,12905,10607,19259,18688,1380,1901,212,1101,6596,198,194,22059,1089,35,19,78,41,1101,97,21,64,91,19,61,88,19,59,1380,9641,1089,661,1380,1434,8789,5960,1382,5108,13,0,141052,76836,23190,0,5416,6922,6731,6360,5618,5150,1
400003,201808,TRAIN_000003,1,1,0,0,0,6172,6172,7866,1,0,3,0,1,0,0,18537,17898,20309,14174,1442,4131,5800,-162,9887,8393,11144,22131,15264,1046,2238,77,647,4267,58,47,2382,0,10,12,45,24,0,35,8,31,26,6,27,19,5,23,208,9504,875,647,0,0,8650,5044,0,875,5,0,112657,67354,18052,0,873,1162,1266,1195,857,462,0
400004,201808,TRAIN_000004,1,1,0,0,0,2484,0,0,1,0,-2,0,0,0,0,0,1498,0,288,0,0,1797,-553,0,0,0,886,0,0,0,-1,0,98,-1,-1,0,0,3,0,0,0,0,16,-2,0,16,-2,0,15,-2,0,0,1498,0,0,0,0,0,0,0,0,0,2,300,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,201808,TRAIN_399995,1,1,0,0,0,2109,0,0,1,0,4,0,0,2,1,0,0,15,0,0,0,0,-882,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,0,11,1,0,11,1,0,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1
799996,201808,TRAIN_399996,1,1,0,0,0,39037,39037,36906,1,0,69,0,1,4,3,21397,17435,17424,139800,9993,11044,80197,39643,21397,14246,29149,54379,15794,2017,3799,282,920,26123,282,282,7599,903,28,11,104,43,2106,132,21,69,132,21,69,132,21,69,844,3188,0,903,0,0,0,1629,0,2106,17,0,185411,75247,18422,0,5066,6085,10259,4310,4747,12113,1
799997,201808,TRAIN_399997,1,1,1,1,1,16916,16916,15126,1,0,29,0,1,0,0,12607,6771,9030,51917,4701,4516,25268,11208,11096,5243,12849,30781,7659,2049,2131,145,401,7269,145,139,3488,313,25,12,78,34,0,64,11,26,64,11,26,60,11,25,1779,1528,0,401,1779,697,1511,1371,0,1118,24,4,43187,21427,6718,0,3213,3468,3554,2243,2105,845,1
799998,201808,TRAIN_399998,0,0,1,1,1,0,0,0,0,0,-2,0,0,2,1,902,0,0,0,0,0,0,-998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2,0,0,-2,0,0,-2,0,0,0,0,0,0,0,0,902,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# vif 표를 출력한 후 vif 계수가 높다고 생각되는 컬럼들을 추가하여 계속 돌려보요요
# vif_X = train_df2.drop(['Transported'], axis=1)
# vif_X = train_df2.drop(['Transported', 'NumberInGroup'], axis=1)
vif_X21 = train_df4.drop(['ID', '기준년월'], axis=1)

# vif를 계산하기 위해 상수항을 추가한다.
vif_X22 = sm.add_constant(vif_X21)

vif_X22

,const,이용가능여부_해외겸용_본인,보유여부_해외겸용_본인,수신거부여부_TM,수신거부여부_메일,수신거부여부_DM,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,이용카드수_신용체크,_2순위카드이용금액,_1순위카드이용건수,_2순위카드이용건수,이용카드수_신용,상향가능한도금액,상향가능CA한도금액,정상청구원금_B5M,정상청구원금_B0M,정상청구원금_B2M,이용금액_일시불_R12M,이용금액_일시불_B0M,이용금액_오프라인_B0M,이용금액_일시불_R6M,이용금액_일시불_R3M,정상입금원금_B5M,정상입금원금_B0M,이용금액_오프라인_R3M,이용금액_오프라인_R6M,정상입금원금_B2M,_3순위업종_이용금액,_2순위업종_이용금액,이용건수_신용_R12M,_2순위쇼핑업종_이용금액,최대이용금액_일시불_R12M,이용건수_신판_R12M,이용건수_일시불_R12M,_1순위업종_이용금액,_3순위쇼핑업종_이용금액,이용가맹점수,이용건수_오프라인_B0M,이용건수_오프라인_R6M,이용건수_오프라인_R3M,쇼핑_도소매_이용금액,이용건수_신용_R6M,이용건수_신용_B0M,이용건수_신용_R3M,이용건수_신판_R6M,이용건수_신판_B0M,이용건수_신판_R3M,이용건수_일시불_R6M,이용건수_일시불_B0M,이용건수_일시불_R3M,_1순위교통업종_이용금액,연체입금원금_B0M,쇼핑_마트_이용금액,쇼핑_슈퍼마켓_이용금액,교통_주유이용금액,이용금액_온라인_B0M,연체입금원금_B5M,연체입금원금_B2M,이용금액_페이_온라인_B0M,_1순위쇼핑업종_이용금액,연속유실적개월수_기본_24M_카드,이용금액대,청구금액_R6M,청구금액_R3M,청구금액_B0,할인건수_R3M,월중평잔_일시불_B0M,월중평잔_일시불,평잔_일시불_3M,잔액_일시불_B0M,잔액_일시불_B1M,잔액_일시불_B2M,인입횟수_ARS_R6M
400000,1.0,0,0,0,0,0,-1190,-1190,2529,1,0,23,0,1,0,0,15593,15008,15712,22754,905,3883,7041,1250,11161,7685,4852,13568,15592,734,1523,159,0,5350,140,137,2000,0,5,5,52,17,0,74,7,23,64,5,18,62,5,17,204,7322,0,0,0,0,4431,120,0,0,13,0,119419,53336,15756,0,1092,1388,1736,780,997,736,0
400001,1.0,0,0,0,0,0,13130,13130,13300,1,0,48,0,1,2,1,3462,3117,2620,52593,3047,4119,20557,14062,2799,3117,15654,16305,2620,1798,2206,193,455,12485,191,191,2504,313,33,14,91,53,662,73,14,41,73,14,41,73,14,41,1281,0,0,455,1281,0,662,0,0,662,12,1,14626,9194,3209,0,2850,3096,4643,2898,2571,6795,1
400002,1.0,1,1,0,0,0,24421,24421,25118,1,0,51,0,1,0,0,23327,22547,24649,54220,6662,9989,38662,18936,14538,12905,10607,19259,18688,1380,1901,212,1101,6596,198,194,22059,1089,35,19,78,41,1101,97,21,64,91,19,61,88,19,59,1380,9641,1089,661,1380,1434,8789,5960,1382,5108,13,0,141052,76836,23190,0,5416,6922,6731,6360,5618,5150,1
400003,1.0,1,1,0,0,0,6172,6172,7866,1,0,3,0,1,0,0,18537,17898,20309,14174,1442,4131,5800,-162,9887,8393,11144,22131,15264,1046,2238,77,647,4267,58,47,2382,0,10,12,45,24,0,35,8,31,26,6,27,19,5,23,208,9504,875,647,0,0,8650,5044,0,875,5,0,112657,67354,18052,0,873,1162,1266,1195,857,462,0
400004,1.0,1,1,0,0,0,2484,0,0,1,0,-2,0,0,0,0,0,1498,0,288,0,0,1797,-553,0,0,0,886,0,0,0,-1,0,98,-1,-1,0,0,3,0,0,0,0,16,-2,0,16,-2,0,15,-2,0,0,1498,0,0,0,0,0,0,0,0,0,2,300,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1.0,1,1,0,0,0,2109,0,0,1,0,4,0,0,2,1,0,0,15,0,0,0,0,-882,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,0,11,1,0,11,1,0,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1
799996,1.0,1,1,0,0,0,39037,39037,36906,1,0,69,0,1,4,3,21397,17435,17424,139800,9993,11044,80197,39643,21397,14246,29149,54379,15794,2017,3799,282,920,26123,282,282,7599,903,28,11,104,43,2106,132,21,69,132,21,69,132,21,69,844,3188,0,903,0,0,0,1629,0,2106,17,0,185411,75247,18422,0,5066,6085,10259,4310,4747,12113,1
799997,1.0,1,1,1,1,1,16916,16916,15126,1,0,29,0,1,0,0,12607,6771,9030,51917,4701,4516,25268,11208,11096,5243,12849,30781,7659,2049,2131,145,401,7269,145,139,3488,313,25,12,78,34,0,64,11,26,64,11,26,60,11,25,1779,1528,0,401,1779,697,1511,1371,0,1118,24,4,43187,21427,6718,0,3213,3468,3554,2243,2105,845,1
799998,1.0,0,0,1,1,1,0,0,0,0,0,-2,0,0,2,1,902,0,0,0,0,0,0,-998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2,0,0,-2,0,0,-2,0,0,0,0,0,0,0,0,902,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1


In [8]:
# vif를 계산한다.
vif2 = pd.DataFrame()
vif2['변수'] = vif_X22.columns
vif2['VIF'] = [variance_inflation_factor(vif_X22.values, i) for i in range(vif_X22.shape[1])]
vif2

,변수,VIF
0,const,58.146018
1,이용가능여부_해외겸용_본인,7.342472
2,보유여부_해외겸용_본인,7.047936
3,수신거부여부_TM,3.441712
4,수신거부여부_메일,10.299614
...,...,...
70,평잔_일시불_3M,19.630350
71,잔액_일시불_B0M,22.533720
72,잔액_일시불_B1M,18.019167
73,잔액_일시불_B2M,7.909232


In [9]:
pd.set_option('display.max_rows', None)
vif_sorted2 = vif2.sort_values(by='VIF', ascending=False)
display(vif_sorted2)

,변수,VIF
46,이용건수_신판_R6M,139002.285239
43,이용건수_신용_R6M,110501.666633
34,이용건수_신판_R12M,100452.336850
48,이용건수_신판_R3M,90318.189964
31,이용건수_신용_R12M,86382.433543
45,이용건수_신용_R3M,61991.192323
51,이용건수_일시불_R3M,27834.289508
49,이용건수_일시불_R6M,27648.920630
47,이용건수_신판_B0M,25253.362524
44,이용건수_신용_B0M,13716.815251


In [11]:
vif_sorted2.to_csv('VIF2.csv', index=False, encoding='utf-8-sig')